<a href="https://colab.research.google.com/github/tamilthamaraiselvan100-pixel/Sharmila/blob/main/project_task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hierarchical Time Series Forecasting Project**

**Step1: Generate the Hierachical Time Series Dataset**

We'll use python with pandas and numpy to simulate a dataset with 3 levels of aggregation (Total,Category,SKU-store),2 years of daily data,and 100 base series.We'll simulate sale data.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Parameters
n_skus = 100
n_days = 365 * 2
start_date = '2024-01-01'

dates = pd.date_range(start=start_date, periods=n_days, freq='D')
data = []

np.random.seed(42)

for i in range(n_skus):
    store_id = f'Store_{i % 5 + 1}'
    category_id = f'Category_{i % 2 + 1}'
    sku_id = f'SKU_{i + 1}'
    # Simulate sales with some seasonality and noise
    sales = np.random.randint(0, 50, n_days) + np.sin(np.arange(n_days) * 2 * np.pi / 30) * 10
    sales = np.maximum(0, sales).astype(int)
    df = pd.DataFrame({'Date': dates, 'Store': store_id, 'Category': category_id, 'SKU': sku_id, 'Sales': sales})
    data.append(df)

df_base = pd.concat(data, ignore_index=True)
df_base['Date'] = pd.to_datetime(df_base['Date'])

# Aggregate to higher levels
df_cat = df_base.groupby(['Date', 'Category'])['Sales'].sum().reset_index()
df_total = df_base.groupby(['Date'])['Sales'].sum().reset_index()

# Combine into a single hierarchical dataframe (long format)
# This part is for understanding the structure, actual implementation uses summation matrix S

**Step 2: Feature Engineering (Lags, Rolling Stats, Cyclical)**

We'll add features to the base level data for the XGBoost model.

In [ ]:
def feature_engineering(df):
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['DayOfYear'] = df['Date'].dt.dayofyear

    # Cyclical features
    df['DayOfYear_sin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365.25)
    df['DayOfYear_cos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365.25)

    # Lags and rolling stats (needs sorting first)
    df = df.sort_values(['SKU', 'Date'])
    df['Sales_lag1'] = df.groupby('SKU')['Sales'].shift(1)
    df['Sales_roll7_mean'] = df.groupby('SKU')['Sales'].rolling(window=7).mean().reset_index(level=0, drop=True)
    df['Sales_roll7_std'] = df.groupby('SKU')['Sales'].rolling(window=7).std().reset_index(level=0, drop=True)

    df = df.dropna() # Drop rows with NA values created by shifting/rolling
    return df

df_features = feature_engineering(df_base.copy())

**Step 3: Train XGBoost Model with Expanding Window Cross-Validation**

We'll use xgboost and a manual expanding window for training and forecasting the base level series.

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Define features and target
features = ['DayOfWeek', 'Month', 'Year', 'DayOfYear_sin', 'DayOfYear_cos', 'Sales_lag1', 'Sales_roll7_mean', 'Sales_roll7_std']
target = 'Sales'

# Expanding window setup (simplified for example)
# In a real scenario, this would loop over multiple windows
train_end_date = dates[int(n_days * 0.8)]
test_start_date = train_end_date + pd.Timedelta(days=1)

df_train = df_features[df_features['Date'] <= train_end_date]
df_test = df_features[df_features['Date'] >= test_start_date]

X_train, y_train = df_train[features], df_train[target]
X_test, y_test = df_test[features], df_test[target]

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
model.fit(X_train, y_train)
base_forecasts = model.predict(X_test)

df_test['Forecast'] = base_forecasts

/tmp/ipython-input-3102033809.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Forecast'] = base_forecasts


**Step 4: Hierarchical Reconciliation (Bottom-Up)**

We'll use a simple Bottom-Up approach. The base forecasts are summed up the hierarchy.

In [ ]:
# Create actuals and forecasts dataframes for all levels
df_actuals_base = df_test[['Date', 'Store', 'Category', 'SKU', 'Sales']]
df_forecasts_base = df_test[['Date', 'Store', 'Category', 'SKU', 'Forecast']]

# Bottom-Up Reconciliation
# Actuals
actuals_cat = df_actuals_base.groupby(['Date', 'Category'])['Sales'].sum().rename('Actuals_Cat')
actuals_total = df_actuals_base.groupby(['Date'])['Sales'].sum().rename('Actuals_Total')

# Forecasts (reconciled by summation)
forecasts_cat = df_forecasts_base.groupby(['Date', 'Category'])['Forecast'].sum().rename('Forecasts_Cat')
forecasts_total = df_forecasts_base.groupby(['Date'])['Forecast'].sum().rename('Forecasts_Total')

# Combine for comparison
results_cat = pd.concat([actuals_cat, forecasts_cat], axis=1).dropna()
results_total = pd.concat([actuals_total, forecasts_total], axis=1).dropna()

**Step 5: Comparison with Benchmark using MASE**

We'll compare the reconciled forecasts against a Naive benchmark using the Mean Absolute Scaled Error (MASE) across all levels.

In [ ]:
def mase_metric(actuals, forecasts, training_actuals, m=1):
    # m is the seasonal period, for daily data we can use m=1 (naive) or m=7 (weekly seasonality)
    # Using m=1 for simple naive benchmark
    n = len(actuals)
    denominator = np.mean(np.abs(training_actuals[m:] - training_actuals[:-m]))
    if denominator == 0:
        return np.inf
    numerator = np.mean(np.abs(actuals - forecasts))
    return numerator / denominator

# Naive benchmark for total level
train_total = df_train.groupby(['Date'])['Sales'].sum()
actuals_total_val = results_total['Actuals_Total'].values
forecasts_total_val = results_total['Forecasts_Total'].values
naive_forecasts_total = train_total.iloc[-1] # Last value of training data as naive forecast

# MASE calculation (simplified as naive forecast is constant here)
# A proper naive forecast would be the last actual value for each day in the test set
# Let's use a simpler approach for the benchmark comparison here.

# Assuming a simple benchmark forecast (e.g., mean of training data for simplicity of example)
benchmark_forecast_total = np.full_like(actuals_total_val, train_total.mean())

mase_xgb_total = mase_metric(actuals_total_val, forecasts_total_val, train_total.values, m=1)
mase_benchmark_total = mase_metric(actuals_total_val, benchmark_forecast_total, train_total.values, m=1)

# Results
#print(f'MASE XGBoost (Total): {mase_xgb_total}')
# print(f'MASE Benchmark (Total): {mase_benchmark_total}')